# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2022 9:37AM,239415,19,ACG-2102490310,ACG North America LLC,Released
1,Jun 13 2022 9:35AM,239414,10,0085755723,ISDIN Corporation,Released
2,Jun 13 2022 9:35AM,239414,10,0085755518,ISDIN Corporation,Released
3,Jun 13 2022 9:35AM,239414,10,0085755697,ISDIN Corporation,Released
4,Jun 13 2022 9:35AM,239414,10,0085755700,ISDIN Corporation,Released
5,Jun 13 2022 9:35AM,239414,10,0085755706,ISDIN Corporation,Released
6,Jun 13 2022 9:35AM,239414,10,0085755705,ISDIN Corporation,Released
7,Jun 13 2022 9:35AM,239414,10,0085755699,ISDIN Corporation,Released
8,Jun 13 2022 9:35AM,239414,10,0085755709,ISDIN Corporation,Released
9,Jun 13 2022 9:35AM,239414,10,0085755713,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
18,239408,Released,3
19,239410,Released,1
20,239413,Released,51
21,239414,Released,56
22,239415,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
239408,9.0,3.0
239410,NaN,1.0
239413,NaN,51.0
239414,NaN,56.0
239415,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
239387,18.0,3.0
239388,0.0,1.0
239389,0.0,1.0
239391,3.0,12.0
239392,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
239387,18,3
239388,0,1
239389,0,1
239391,3,12
239392,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,239387,18,3
1,239388,0,1
2,239389,0,1
3,239391,3,12
4,239392,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,239387,18,3
1,239388,,1
2,239389,,1
3,239391,3,12
4,239392,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2022 9:37AM,239415,19,ACG North America LLC
1,Jun 13 2022 9:35AM,239414,10,ISDIN Corporation
57,Jun 13 2022 9:34AM,239413,10,ISDIN Corporation
108,Jun 13 2022 9:19AM,239410,20,"Exact-Rx, Inc."
109,Jun 13 2022 9:14AM,239408,12,Hush Hush
121,Jun 13 2022 9:14AM,239407,16,Sartorius Stedim North America
122,Jun 13 2022 9:11AM,239403,10,ISDIN Corporation
168,Jun 13 2022 9:10AM,239404,18,"ClearSpec, LLC"
169,Jun 13 2022 9:07AM,239397,10,ISDIN Corporation
219,Jun 13 2022 9:07AM,239395,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 13 2022 9:37AM,239415,19,ACG North America LLC,,1
1,Jun 13 2022 9:35AM,239414,10,ISDIN Corporation,,56
2,Jun 13 2022 9:34AM,239413,10,ISDIN Corporation,,51
3,Jun 13 2022 9:19AM,239410,20,"Exact-Rx, Inc.",,1
4,Jun 13 2022 9:14AM,239408,12,Hush Hush,9,3
5,Jun 13 2022 9:14AM,239407,16,Sartorius Stedim North America,1,
6,Jun 13 2022 9:11AM,239403,10,ISDIN Corporation,,46
7,Jun 13 2022 9:10AM,239404,18,"ClearSpec, LLC",,1
8,Jun 13 2022 9:07AM,239397,10,ISDIN Corporation,,50
9,Jun 13 2022 9:07AM,239395,10,ISDIN Corporation,,50


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2022 9:37AM,239415,19,ACG North America LLC,1,
1,Jun 13 2022 9:35AM,239414,10,ISDIN Corporation,56,
2,Jun 13 2022 9:34AM,239413,10,ISDIN Corporation,51,
3,Jun 13 2022 9:19AM,239410,20,"Exact-Rx, Inc.",1,
4,Jun 13 2022 9:14AM,239408,12,Hush Hush,3,9
5,Jun 13 2022 9:14AM,239407,16,Sartorius Stedim North America,,1
6,Jun 13 2022 9:11AM,239403,10,ISDIN Corporation,46,
7,Jun 13 2022 9:10AM,239404,18,"ClearSpec, LLC",1,
8,Jun 13 2022 9:07AM,239397,10,ISDIN Corporation,50,
9,Jun 13 2022 9:07AM,239395,10,ISDIN Corporation,50,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2022 9:37AM,239415,19,ACG North America LLC,1,
1,Jun 13 2022 9:35AM,239414,10,ISDIN Corporation,56,
2,Jun 13 2022 9:34AM,239413,10,ISDIN Corporation,51,
3,Jun 13 2022 9:19AM,239410,20,"Exact-Rx, Inc.",1,
4,Jun 13 2022 9:14AM,239408,12,Hush Hush,3,9


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2022 9:37AM,239415,19,ACG North America LLC,1.0,NaN
1,Jun 13 2022 9:35AM,239414,10,ISDIN Corporation,56.0,NaN
2,Jun 13 2022 9:34AM,239413,10,ISDIN Corporation,51.0,NaN
3,Jun 13 2022 9:19AM,239410,20,"Exact-Rx, Inc.",1.0,NaN
4,Jun 13 2022 9:14AM,239408,12,Hush Hush,3.0,9.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2022 9:37AM,239415,19,ACG North America LLC,1.0,0.0
1,Jun 13 2022 9:35AM,239414,10,ISDIN Corporation,56.0,0.0
2,Jun 13 2022 9:34AM,239413,10,ISDIN Corporation,51.0,0.0
3,Jun 13 2022 9:19AM,239410,20,"Exact-Rx, Inc.",1.0,0.0
4,Jun 13 2022 9:14AM,239408,12,Hush Hush,3.0,9.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1915205,282.0,3.0
12,239408,3.0,9.0
16,239407,0.0,1.0
18,957594,4.0,0.0
19,478802,4.0,18.0
20,478808,15.0,0.0
22,478777,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1915205,282.0,3.0
1,12,239408,3.0,9.0
2,16,239407,0.0,1.0
3,18,957594,4.0,0.0
4,19,478802,4.0,18.0
5,20,478808,15.0,0.0
6,22,478777,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,282.0,3.0
1,12,3.0,9.0
2,16,0.0,1.0
3,18,4.0,0.0
4,19,4.0,18.0
5,20,15.0,0.0
6,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,282.0
1,12,Released,3.0
2,16,Released,0.0
3,18,Released,4.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,22
Status,,,,,,,
Executing,3.0,9.0,1.0,0.0,18.0,0.0,0.0
Released,282.0,3.0,0.0,4.0,4.0,15.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,22
0,Executing,3.0,9.0,1.0,0.0,18.0,0.0,0.0
1,Released,282.0,3.0,0.0,4.0,4.0,15.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,22
0,Executing,3.0,9.0,1.0,0.0,18.0,0.0,0.0
1,Released,282.0,3.0,0.0,4.0,4.0,15.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()